<a href="https://colab.research.google.com/github/hj245668/DL/blob/main/1211_lyricsGen_cpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한글 가사 생성기 만들기 튜토리얼 (TensorFlow + LSTM)

**가사 데이터를 기반으로 가사 생성**을 진행하는 과정을 정리하였습니다.

1. 환경 설정 및 라이브러리 불러오기  
2. 데이터 준비 (텍스트 파일에서 한 줄씩 읽어오기)  
3. 전처리: `<start>`, `<end>` 토큰 추가 및 정규식 처리  
4. 토크나이저로 정수 시퀀스 만들기 + 패딩 처리  
5. 학습/검증 데이터셋 분리 및 `tf.data.Dataset` 구성  
6. LSTM 기반 `TextGenerator` 모델 정의  
7. 모델 학습 (`model.fit`)  
8. 가사 생성 함수 구현 및 테스트  

## 1. 환경 설정 및 라이브러리 불러오기

이 단계에서는 가사 생성 모델을 학습하는 데 필요한 기본 라이브러리를 불러옵니다.

- `os`: 파일/폴더 경로 처리
- `re`: 정규표현식(텍스트 전처리)
- `numpy`: 수치 계산
- `tensorflow`: 딥러닝 라이브러리 (LSTM, Embedding, Dataset 등)

> 만약 로컬 환경에서 실행하는데 `tensorflow`가 설치되어 있지 않다면,  
> 터미널 또는 노트북 셀에서 `pip install tensorflow` 명령을 먼저 실행해야 합니다.


In [1]:
import os
import re
import numpy as np
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


## 2. 데이터 준비: 텍스트 파일에서 한 줄씩 읽어오기

이 예제에서는 **`data/lyrics/` 폴더 아래에 여러 개의 txt 파일**이 있고,  
각 파일이 다음과 같은 형태라고 가정합니다.

```text
첫 번째 노래의 가사 1줄
첫 번째 노래의 가사 2줄

공백 줄은 자동으로 제거됩니다.
```

즉, **한 줄 = 한 문장(또는 한 소절)** 구조입니다.

coding

1. `data/lyrics` 폴더 안의 `.txt` 파일을 모두 찾음  
2. 각 파일을 열어서 `read().splitlines()`로 줄 단위로 나눔  
3. 공백 줄(`""` 또는 공백만 있는 줄)은 제거  
4. 모든 줄을 하나의 리스트 `raw_corpus`에 모음  


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 사용자가 자신의 환경에 맞게 바꿔야 하는 부분: 가사 데이터가 있는 폴더 경로
data_dir = "/content/drive/MyDrive/SQL_modu/1211_ML"

raw_corpus = []

if not os.path.isdir(data_dir):
    print(f"경고: '{data_dir}' 폴더가 존재하지 않습니다. 경로를 수정하세요.")
else:
    for fname in os.listdir(data_dir):
        if not fname.endswith(".txt"):
            continue
        file_path = os.path.join(data_dir, fname)
        with open(file_path, encoding="utf-8") as f:
            lines = f.read().splitlines()
            # 양쪽 공백 제거 후, 완전히 빈 줄은 제외
            lines = [line.strip() for line in lines if line.strip() != ""]
            raw_corpus.extend(lines)

    print("전체 문장(줄) 개수:", len(raw_corpus))
    print("\n[샘플 5줄 미리 보기]\n")
    for line in raw_corpus[:5]:
        print(line)

전체 문장(줄) 개수: 32777

[샘플 5줄 미리 보기]

First Citizen:
Before we proceed any further, hear me speak.
All:
Speak, speak.
First Citizen:


## 3. 전처리: `<start>`, `<end>` 토큰 추가 및 정규식 처리

딥러닝 모델에 텍스트를 넣기 전에 **전처리(preprocessing)**를 해 줍니다.

여기서 하는 주요 작업은:

1. 양쪽 공백 제거 (`strip`)  
2. `? . ! ,` 등의 문장부호 앞뒤에 공백을 넣어 토큰 분리가 쉬워지도록 함  
3. 여러 개의 공백을 하나로 줄이기  
4. **허용된 문자만 남기고 나머지는 공백으로 치환**  
   - 한글: `가-힣`  
   - 영문: `a-zA-Z`  
   - 숫자: `0-9`  
   - 기본 문장부호: `? . ! ,`  
5. 문장 앞뒤에 `<start>`, `<end>` 토큰을 붙여서 **문장의 시작과 끝을 명시**

이렇게 하면, 나중에 모델이 **문장이 어디서 시작해서 어디서 끝나는지**를 배울 수 있습니다.


In [4]:
def preprocess_sentence_ko(sentence: str) -> str:
    # 1) 양쪽 공백 제거
    sentence = sentence.strip()
    # 2) 문장부호 앞뒤로 공백 추가
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    # 3) 여러 공백을 하나로 통일
    sentence = re.sub(r"\s+", " ", sentence)
    # 4) 허용되지 않는 문자는 공백으로 치환
    #    한글(가-힣), 영문, 숫자, 일부 문장부호만 남긴다.
    sentence = re.sub(r"[^0-9a-zA-Z가-힣?.!,]+", " ", sentence)
    # 5) 다시 양쪽 공백 제거
    sentence = sentence.strip()
    # 6) 시작/끝 토큰 추가
    sentence = "<start> " + sentence + " <end>"
    return sentence


# 전처리 적용
corpus = [preprocess_sentence_ko(s) for s in raw_corpus]

print("전처리 후 문장 수:", len(corpus))
print("\n[전처리된 문장 5개]\n")
for line in corpus[:5]:
    print(line)

전처리 후 문장 수: 32777

[전처리된 문장 5개]

<start> First Citizen <end>
<start> Before we proceed any further , hear me speak . <end>
<start> All <end>
<start> Speak , speak . <end>
<start> First Citizen <end>


## 4. 토크나이저로 정수 시퀀스 만들기 + 패딩 처리

신경망은 텍스트 문자열을 직접 처리할 수 없으므로, **각 단어를 정수 ID로 바꾸는 과정**이 필요합니다.

### 주요 개념

- `Tokenizer`  
  - 단어 → 정수, 정수 → 단어 매핑을 관리하는 도구입니다.
  - `num_words`: 사용할 최대 단어 수 (자주 등장하는 단어부터 순서대로 사용)
  - `oov_token`: 사전에 없는(out-of-vocabulary) 단어가 나왔을 때 대신 사용할 토큰

- `texts_to_sequences`  
  - 각 문장을 `[정수, 정수, 정수, ...]` 형태의 리스트로 변환

- `pad_sequences`  
  - 문장별 길이가 다르기 때문에, 모델에 넣기 위해 **모든 문장을 같은 길이로 맞추는 작업**입니다.
  - `maxlen`보다 짧은 문장은 뒤를 0으로 채우고, 더 긴 문장은 잘라냅니다.

아래 코드에서는:

1. `Tokenizer`를 정의하고, `corpus`에 대해 `fit_on_texts`를 수행  
2. `texts_to_sequences`로 정수 시퀀스 생성  
3. `pad_sequences`로 길이를 `maxlen`으로 맞춤  


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 사용할 최대 단어 수
NUM_WORDS = 12000
# 문장 최대 길이 (데이터에 따라 조정 가능)
MAX_LEN = 30

def tokenize(corpus):
    tokenizer = Tokenizer(
        num_words=NUM_WORDS,
        filters=" ",        # 이미 정규식으로 전처리를 했기 때문에 단순 공백만 필터
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = pad_sequences(
        tensor,
        padding="post",     # 뒤에서부터 0으로 채움
        maxlen=MAX_LEN,
    )
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

print("텐서(shape):", tensor.shape)
print("vocab size(단어 수):", len(tokenizer.word_index) + 1)

텐서(shape): (32777, 30)
vocab size(단어 수): 11464


## 5. 학습/검증 데이터셋 분리 및 `tf.data.Dataset` 구성

모델의 성능을 객관적으로 평가하려면 **훈련에 사용하지 않은 데이터(검증 데이터)**가 필요합니다.

여기에서는 간단하게:

- 앞쪽 80%: 학습용
- 뒤쪽 20%: 검증용

으로 나누겠습니다.

또한, 시퀀스 모델을 학습하기 위해 **입력과 라벨을 한 토큰씩 밀어서 구성**합니다.

- 입력(`decoder_input`): `[w1, w2, w3, ..., w_{n-1}]`  
- 라벨(`decoder_label`): `[w2, w3, w4, ..., w_n]`  

이렇게 하면, 모델이 “현재까지 등장한 단어들을 보고 다음 단어를 예측”하도록 학습됩니다.


In [6]:
# 데이터 섞기
np.random.seed(42)
indices = np.arange(tensor.shape[0])
np.random.shuffle(indices)
tensor = tensor[indices]

# 80% 학습, 20% 검증
split_at = int(tensor.shape[0] * 0.8)
train_seq = tensor[:split_at]
val_seq = tensor[split_at:]

# 디코더 입력/라벨 분리
# 입력: 마지막 토큰 제외
# 라벨: 첫 번째 토큰 제외
dec_train = train_seq[:, :-1]
label_train = train_seq[:, 1:]
dec_val = val_seq[:, :-1]
label_val = val_seq[:, 1:]

print("학습 dec/label:", dec_train.shape, label_train.shape)
print("검증 dec/label:", dec_val.shape, label_val.shape)

# tf.data.Dataset 구성
BATCH_SIZE = 64
BUFFER_SIZE = 10000

train_dataset = tf.data.Dataset.from_tensor_slices((dec_train, label_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

val_dataset = tf.data.Dataset.from_tensor_slices((dec_val, label_val))
val_dataset = val_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

train_dataset, val_dataset

학습 dec/label: (26221, 29) (26221, 29)
검증 dec/label: (6556, 29) (6556, 29)


(<_BatchDataset element_spec=(TensorSpec(shape=(64, 29), dtype=tf.int32, name=None), TensorSpec(shape=(64, 29), dtype=tf.int32, name=None))>,
 <_BatchDataset element_spec=(TensorSpec(shape=(64, 29), dtype=tf.int32, name=None), TensorSpec(shape=(64, 29), dtype=tf.int32, name=None))>)

## 6. LSTM 기반 `TextGenerator` 모델 정의

여기서는 가장 전형적인 **Embedding + LSTM 2층 + Dense** 구조를 사용합니다.

구성은 다음과 같습니다.

1. `Embedding`  
   - 정수로 표현된 단어 ID → 밀집 벡터(임베딩)로 변환  
   - 단어 간 의미/유사도를 벡터 공간에서 학습

2. 첫 번째 `LSTM` (return_sequences=True)  
   - 시퀀스를 입력받아 시퀀스를 출력 (각 타임스텝별 hidden state)

3. 두 번째 `LSTM` (return_sequences=True)  
   - 더 복잡한 패턴 학습, 상위 레벨 시퀀스 표현

4. `Dense(vocab_size)`  
   - 각 타임스텝에서 **전체 vocabulary 크기만큼의 로짓(logits)**을 출력  
   - Softmax를 통해 다음 단어의 확률분포로 해석 가능

> 여기서는 `tf.keras.Model`을 상속받아 `TextGenerator` 클래스를 만듭니다.


In [7]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        # x: (batch_size, seq_len)
        out = self.embedding(x)   # (batch_size, seq_len, embedding_size)
        out = self.rnn_1(out)     # (batch_size, seq_len, hidden_size)
        out = self.rnn_2(out)     # (batch_size, seq_len, hidden_size)
        out = self.linear(out)    # (batch_size, seq_len, vocab_size)
        return out


vocab_size = len(tokenizer.word_index) + 1

# 하이퍼파라미터 (필요에 따라 조정 가능)
EMBEDDING_SIZE = 256
HIDDEN_SIZE = 512

model = TextGenerator(vocab_size, EMBEDDING_SIZE, HIDDEN_SIZE)

# 더미 입력으로 모델 한 번 호출해서 구조 확인
dummy_x = tf.zeros((1, MAX_LEN - 1), dtype=tf.int32)
dummy_y = model(dummy_x)

print("모델 출력 shape:", dummy_y.shape)

모델 출력 shape: (1, 29, 11464)


## 7. 모델 컴파일 및 학습

### 손실 함수

- `SparseCategoricalCrossentropy(from_logits=True)`  
  - 다중 클래스 분류(여기서는 ‘다음 단어’ 예측)에 자주 사용되는 손실 함수입니다.
  - `from_logits=True`로 설정하여 모델의 출력이 Softmax를 거치지 않은 **로짓(logits)**임을 명시합니다.

### 옵티마이저

- `Adam`  
  - 학습 속도와 안정성이 좋아 NLP에서 널리 사용되는 옵티마이저입니다.

### 학습 설정

- `epochs`  
  - 전체 데이터를 몇 번 반복해서 학습할지 정합니다.
  - 데이터 양과 모델 크기에 따라 5~30 사이에서 조정해보면 좋습니다.

> 주의: GPU가 없는 환경에서는 epoch 수를 너무 크게 하면 시간이 오래 걸릴 수 있습니다.


In [8]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction="none",   # 각 타임스텝별 손실을 그대로 반환 (마스크 등 응용 가능)
)

optimizer = tf.keras.optimizers.Adam()

model.compile(loss=loss, optimizer=optimizer)

EPOCHS = 5  # 시작은 작게, 결과 보고 늘리는 것을 추천

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS
)

Epoch 1/5
409/409 ━━━━━━━━━━━━━━━━━━━━ 2185s 5s/step - loss: 2.5598 - val_loss: 1.6831
Epoch 2/5
409/409 ━━━━━━━━━━━━━━━━━━━━ 2190s 5s/step - loss: 1.6288 - val_loss: 1.5773
Epoch 3/5
409/409 ━━━━━━━━━━━━━━━━━━━━ 2171s 5s/step - loss: 1.5359 - val_loss: 1.5351
Epoch 4/5
409/409 ━━━━━━━━━━━━━━━━━━━━ 2468s 6s/step - loss: 1.4684 - val_loss: 1.5087
Epoch 5/5
409/409 ━━━━━━━━━━━━━━━━━━━━ 2481s 6s/step - loss: 1.4263 - val_loss: 1.4926


## 8. 가사 생성 함수 구현 및 테스트

이제 학습된 모델을 사용해 **새로운 가사 문장**을 생성해 보겠습니다.

전체 흐름은 다음과 같습니다.

1. 시작 문장을 `<start>` 토큰과 함께 `tokenizer.texts_to_sequences`로 정수 시퀀스로 변환  
2. 텐서로 바꿔서 모델에 입력  
3. 모델이 출력한 로짓(logits)을 Softmax로 확률 분포로 바꾼 뒤,  
   - 가장 확률이 높은 단어를 `argmax`로 선택 (단순 그리디 전략)  
4. 선택한 단어를 입력 시퀀스 뒤에 붙이고, 다시 모델에 넣어 반복  
5. `<end>` 토큰이 나오거나 최대 길이에 도달하면 종료  
6. 정수 시퀀스를 다시 단어로 바꿔 하나의 문장으로 합침  


In [10]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=30):
    # 1) 시작 문장을 시퀀스로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)

    # 2) <end> 토큰 ID 가져오기
    end_token = tokenizer.word_index.get("<end>")
    if end_token is None:
        raise ValueError("<end> 토큰이 tokenizer에 없습니다. 전처리 과정을 확인하세요.")

    while True:
        # 3) 모델 예측
        predictions = model(test_tensor)  # (1, seq_len, vocab_size)
        predictions = predictions[:, -1, :]  # 마지막 타임스텝의 예측만 사용

        # 4) 확률분포로 변환 후 argmax로 단어 선택 (그리디 전략)
        predicted_id = tf.argmax(tf.nn.softmax(predictions, axis=-1), axis=-1).numpy()[0]

        # 5) 선택한 단어를 시퀀스 뒤에 붙임
        test_tensor = tf.concat(
            [test_tensor, tf.expand_dims([predicted_id], axis=0)],
            axis=-1
        )

        # 6) 종료 조건: <end> 토큰이거나, max_len 도달
        if predicted_id == end_token:
            break
        if test_tensor.shape[1] >= max_len:
            break

    # 7) 정수 시퀀스를 단어로 복원
    generated = ""
    for word_index in test_tensor[0].numpy():
        word = tokenizer.index_word.get(word_index, "<unk>")
        generated += word + " "

    return generated


# 예시: '사랑이란'으로 시작하는 문장 생성
example = generate_text(
    model,
    tokenizer,
    init_sentence="<start> i",
    max_len=30,
)
print(example)

InvalidArgumentError: cannot compute ConcatV2 as input #1(zero-based) was expected to be a int64 tensor but is a int32 tensor [Op:ConcatV2] name: concat

## 9. 마무리 및 커스터마이징 포인트

이 노트북에서 구현한 것은 **한글 가사 데이터로 LSTM 기반 가사 생성기**를 만드는 기본 골격입니다.

### 직접 바꿔보면 좋은 부분

1. **데이터 경로 및 형식**
   - `data_dir = "data/lyrics"` 부분을 자신의 폴더 구조에 맞게 수정
   - 한 파일에 여러 곡이 들어있다면, 구분자를 기준으로 나눠서 `raw_corpus`를 구성해도 됩니다.

2. **전처리 규칙 (`preprocess_sentence_ko`)**
   - 이모지, 해시태그, 괄호 등을 살리고 싶다면 정규식 범위를 늘리면 됩니다.
   - 줄 단위가 아니라 문장 단위로 나누고 싶다면 `.` 또는 `?` 기준으로 split해서 사용해도 됩니다.

3. **모델 크기**
   - `EMBEDDING_SIZE`, `HIDDEN_SIZE`를 늘리면 표현력은 좋아지지만 메모리와 시간이 더 필요합니다.
   - 데이터가 적다면 너무 큰 모델은 과적합(overfitting)을 일으킬 수 있으니 주의하세요.

4. **학습 설정**
   - `EPOCHS` 수를 변경하면서 loss의 변화를 관찰해 보세요.
   - EarlyStopping, ModelCheckpoint 등의 콜백을 추가하면 더 안정적인 학습이 가능합니다.

5. **텍스트 생성 전략**
   - 현재는 가장 확률이 높은 단어만 선택하는 **그리디(탐욕적) 전략**을 사용하고 있습니다.
   - 더 다양한 문장을 얻고 싶다면 temperature sampling, top-k, top-p(nucleus) sampling 등을 적용해 볼 수 있습니다.

---

이 노트북을 기본 틀로 삼아서,  
- 랩 가사, 발라드, 트롯, K-POP, 영어/일본어 등  
원하는 스타일에 맞게 데이터를 넣고 튜닝해 보시면 됩니다.
